导入库

In [1]:
import os.path

import pandas as pd
import numpy as np

import scipy.signal
import scipy.io
import utils

from pynwb import NWBHDF5IO
from tqdm import tqdm
import subprocess


公共变量

In [2]:
winL = 0.05
frameshift = 0.01
modelOrder = 4
stepSize = 5
path_bids = r'./SingleWordProductionDutch-iBIDS'
feat_path = r'./feat'
result_path = r'./res'
# pts = ['sub-%02d'%i for i in range(1,11)]
participants = pd.read_csv(os.path.join(path_bids,'participants.tsv'), delimiter='\t')
pts = participants['participant_id']

数据预处理
- eeg数据
- 语音音频数据
- 语音文本数据

In [5]:
os.makedirs(os.path.join(feat_path), exist_ok=True)
for p_id, pt in enumerate(tqdm(pts)):
    os.makedirs(os.path.join(feat_path,f'{pt}'), exist_ok=True)
    #Load data
    io = NWBHDF5IO(os.path.join(path_bids,pt,'ieeg',f'{pt}_task-wordProduction_ieeg.nwb'), 'r')
    nwbfile = io.read()
    #sEEG
    eeg = nwbfile.acquisition['iEEG'].data[:]
    eeg_sr = 1024
    #audio
    audio = nwbfile.acquisition['Audio'].data[:]
    audio_sr = 48000
    #words (markers)
    words = nwbfile.acquisition['Stimulus'].data[:]
    words = np.array(words, dtype=str)
    io.close()
    #channels
    channels = pd.read_csv(os.path.join(path_bids,pt,'ieeg',f'{pt}_task-wordProduction_channels.tsv'), delimiter='\t')
    channels = np.array(channels['name'])
    
    #Extract HG features
    feat = utils.extractHG(eeg,eeg_sr, windowLength=winL,frameshift=frameshift)
 

    #Process Audio
    target_SR = 16000
    audio = scipy.signal.decimate(audio,int(audio_sr / target_SR))
    audio_sr = target_SR
    scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
    # scaled[np.abs(scaled)<1000] = 0
    record_words = utils.dict4wav(words)
    os.makedirs(os.path.join(feat_path,f'{pt}','audio'), exist_ok=True)
    # os.makedirs(os.path.join(path_output,f'{pt}','result'), exist_ok=True)
    for i in range(len(record_words)):
        audio_length = int(len(scaled)/len(record_words))
        eeg_length = int(len(eeg)/len(record_words))
        word_length = 1
        scipy.io.wavfile.write(os.path.join(feat_path,f'{pt}','audio',f'{pt}_{i}.wav'),audio_sr,scaled[i*audio_length:(i+1)*audio_length])
        np.save(os.path.join(feat_path,f'{pt}','audio',f'{pt}_{i}.npy'), eeg[i*eeg_length:(i+1)*eeg_length])
        with open(os.path.join(feat_path,f'{pt}','audio',f'{pt}_{i}.lab'), "w", encoding="utf-8") as file:
            for word in record_words[i*word_length:(i+1)*word_length]:
                file.write("%s\n" % word)
    #Extract spectrogram
    melSpec = utils.extractMelSpecs(scaled,audio_sr,windowLength=winL,frameshift=frameshift)
    #Align to EEG features
    words = utils.downsampleLabels(words,eeg_sr,windowLength=winL,frameshift=frameshift)
    #adjust length (differences might occur due to rounding in the number of windows)
    if melSpec.shape[0]!=feat.shape[0]:
        tLen = np.min([melSpec.shape[0],feat.shape[0]])
        melSpec = melSpec[:tLen,:]
        feat = feat[:tLen,:]

    #Create feature names by appending the temporal shift 
    feature_names = utils.nameVector(channels[:,None], modelOrder=modelOrder)

    #Save everything
    np.save(os.path.join(feat_path,f'{pt}',f'{pt}_feat.npy'), feat)
    np.save(os.path.join(feat_path,f'{pt}',f'{pt}_procWords.npy'), words)
    np.save(os.path.join(feat_path,f'{pt}',f'{pt}_spec.npy'), melSpec)
    np.save(os.path.join(feat_path,f'{pt}',f'{pt}_feat_names.npy'), feature_names)
    
    
    with open(os.path.join(feat_path,f'{pt}',f'{pt}_words.txt'), "w", encoding="utf-8") as file:
        for word in record_words:
            file.write("%s\n" % word)

100%|██████████| 10/10 [00:51<00:00,  5.17s/it]


使用mfa标注语音数据

In [11]:
for p_id, pt in enumerate(tqdm(participants['participant_id'])):
    pass# subprocess.run(['mfa', 'align',os.path.join(feat_path,f'{pt}','audio'),'dutch_cv','dutch_cv',os.path.join(feat_path,f'{pt}','audio'),'--clean'], stdout=subprocess.PIPE, text=True)

100%|██████████| 10/10 [16:19<00:00, 97.93s/it]


处理语音-音标-eeg数据，并按照音标以npy保存

In [4]:
phones_output_path = os.path.join(feat_path,'phones')
os.makedirs(phones_output_path, exist_ok=True)
big_phones_log = open('./log/big_phones.txt','w')
small_phones_log = open('./log/small_phones.txt','w')
maxlength = 0
minlength = 999
big_phones = 0
small_phones = 0
for pt in pts:
    word_count = 0
    with open(os.path.join(feat_path,f'{pt}',f'{pt}_words.txt'), 'r', encoding='utf-8') as file:
        text = file.read()
        words = text.split()
        word_count = len(words)
    for i in range(word_count):
        phones_info = utils.readTextGridPhones(feat_path,pt,i)
        audio_sample_rate,audio_data = scipy.io.wavfile.read(os.path.join(feat_path,f'{pt}','audio',f'{pt}_{i}.wav'))
        eeg_sample_rate = 1024
        eeg_data = np.load(os.path.join(feat_path,f'{pt}','audio',f'{pt}_{i}.npy'))
        for phone_info in phones_info:
            phone_length = phone_info['end_time']-phone_info['start_time']
            maxlength = max(maxlength,phone_length)
            minlength = min(minlength,phone_length)
            phone_info['audio'] = audio_data[int(phone_info['start_time']*audio_sample_rate):int(phone_info['end_time']*audio_sample_rate)]
            phone_info['eeg'] = eeg_data[int(phone_info['start_time']*eeg_sample_rate):int(phone_info['end_time']*eeg_sample_rate)]
            if not os.path.exists(os.path.join(phones_output_path,phone_info["label"])):
                os.makedirs(os.path.join(phones_output_path,phone_info["label"]))
            count = 1
            save_name = f'{pt}_{i}_{count}.npy'
            while os.path.exists(os.path.join(phones_output_path,phone_info["label"],save_name)):
                count += 1
                save_name = f'{pt}_{i}_{count}.npy'
            if phone_length > 0.6:
                big_phones_log.write(f'{pt}_{i}_{phone_info["label"]}_{count} : {phone_length}\n')
                big_phones += 1
            elif phone_length < 0.025:
                small_phones_log.write(f'{pt}_{i}_{phone_info["label"]}_{count} : {phone_length}\n')
                small_phones += 1
            else:
                np.save(os.path.join(phones_output_path,phone_info["label"],save_name),phone_info)
print('max length:',maxlength)
print('min length:',minlength)

max length: 0.6
min length: 0.02600348744905001


按照人保存语音数据
- 相当于滤去无声、杂音的纯语音数据

In [3]:
words_output_path = os.path.join(feat_path,'words')
os.makedirs(words_output_path, exist_ok=True)
for pt in pts:
    if not os.path.exists(os.path.join(words_output_path,pt)):
        os.makedirs(os.path.join(words_output_path,pt))
    word_count = 0
    with open(os.path.join(feat_path,f'{pt}',f'{pt}_words.txt'), 'r', encoding='utf-8') as file:
        text = file.read()
        words = text.split()
        word_count = len(words)
    for i in range(word_count):
        word_info = utils.readTextGridWords(feat_path,pt,i)
        audio_sample_rate,audio_data = scipy.io.wavfile.read(os.path.join(feat_path,f'{pt}','audio',f'{pt}_{i}.wav'))
        eeg_sample_rate = 1024
        eeg_data = np.load(os.path.join(feat_path,f'{pt}','audio',f'{pt}_{i}.npy'))
        word_info['audio'] = audio_data[int(word_info['start_time']*audio_sample_rate):int(word_info['end_time']*audio_sample_rate)]
        word_info['eeg'] = eeg_data[int(word_info['start_time']*eeg_sample_rate):int(word_info['end_time']*eeg_sample_rate)]
        # save_name = f'{word_info["label"]}.npy'
        save_name = f'{i}.npy'
        np.save(os.path.join(words_output_path,pt,save_name),word_info)
